# Error Handling

Rust 不同于别的语言，使用**枚举**作为错误处理的方式

- 枚举`enum`是一种类型，可以包含多个变量之一

In [ ]:
enum TrafficLightColor {    // Type
    Red,                    // variants
    Yellow,
    Green,
}

let current_state = TrafficLightColor::Green;

- Rust: `match`表达式类似于 C/C++/Java 中的 `switch` 语句，但是必须覆盖所有可能的值

如果没有全部覆盖，则会报错，如下所示

In [3]:
enum TrafficLightColor {    // Type
    Red,                    // variants
    Yellow,
    Green,
}


fn drive(light_state: TrafficLightColor) {
    match light_state {
        TrafficLightColor::Green => println!("zoom zoom!"),
        TrafficLightColor::Red => println!("sitting like a boulder!"),
    }
}

Error: non-exhaustive patterns: `TrafficLightColor::Yellow` not covered

- 如果你只对其中几个感兴趣，可以使用默认绑定来捕获所有其他情况。

```rust
match light_state {
    TrafficLightColor::Green => println!("zoom zoom!"),
    _ => println!("do not pass go"), // default binding. 
    // DO THIS in all other cases
}

- 与大多数语言的枚举不同，Rust 枚举可以存储任意数据

In [ ]:
enum Location {
    Coordinates(f32, f32),
    Address(String),
    Unknown,
}

比如：
想要存储某物的位置，并希望有选项表示它：
- 经纬度坐标（32位浮点数）
- 地址（值为字符串）
- 未知（没有任何关联数据）

可以使用 `match` 表达式从变量中提取数据

In [5]:
enum Location {
    Coordinates(f32, f32),
    Address(String),
    Unknown,
}

fn print_location(loc: Location) {
    match loc {
        Location::Coordinates(lat, long) => {
            println!("Person is at ({}, {})", lat, long);
        },
        Location::Address(addr) => {
            println!("Person is at {}", addr);
        },
        Location::Unknown => println!("Location unknown!"),
    }
}

print_location(Location::Address("yeugdauabi233!".to_string()));

Person is at yeugdauabi233!


如果使用枚举来清晰地表示成功返回/错误会怎样呢？

- 如果函数成功运行，则返回OK (任何返回值)
- 如果发生错误，则返回Err (某个错误对象)

In [ ]:
// Rust 标准库中的一部分
enum Result<T, E> {
    OK(T),
    Err(E),
}

// T 是成功时要存储在OK中的值的类型
// E 是错误时要存储在Err中的值的类型

In [ ]:
// u32 成功返回的类型，str 错误返回的类型
fn gen_num_sometimes() -> Result<u32, &'stack str> {
    if get_random_num() > 10 {
        OK(get_random_num())
    } else {
        Err("number is less than 11")
    }
}

fn main() {
    match gen_num_sometimes() {
        OK(num) => println!("get number: {}", num),
        Err(message) => println!("Operation failied: {}", message),
    }
}